<a href="https://colab.research.google.com/github/iamDivyanshi/Python_Assignment/blob/main/Travel-Destination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


uploaded = files.upload()

Saving Travel details dataset.csv to Travel details dataset.csv


In [53]:
data = pd.read_csv('Travel details dataset.csv')

In [54]:
def standardize_date(date_str):
    if pd.isna(date_str):  # Check if the date string is missing
        return None
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%Y-%m-%d')
    except:
        try:
            return pd.to_datetime(date_str, format='%m-%d-%Y').strftime('%Y-%m-%d')
        except:
            return None  # Return None if both formats fail

data['Start date'] = data['Start date'].apply(standardize_date)
data['End date'] = data['End date'].apply(standardize_date)

In [56]:
def clean_currency(value):
    if isinstance(value, str):
        value = value.replace('USD', '').replace('$', '').replace('€', '').strip()
        # Remove commas before converting to float
        value = value.replace(',', '')
    return float(value) if value else np.nan

data['Accommodation cost'] = data['Accommodation cost'].apply(clean_currency)
data['Transportation cost'] = data['Transportation cost'].apply(clean_currency)

In [57]:
data.replace('########', np.nan, inplace=True)
data.replace('...', np.nan, inplace=True)

In [58]:
data['Accommodation cost'].fillna(0, inplace=True)
data['Transportation cost'].fillna(0, inplace=True)
data['Transportation type'].fillna('Unknown', inplace=True)

In [59]:
country_dict = {
    "Paris": "Paris, France",
    "Bali": "Bali, Indonesia",
    "London": "London, UK",
    "Tokyo": "Tokyo, Japan",
    "New York": "New York, USA",
    "Sydney": "Sydney, Australia",
    "Barcelona": "Barcelona, Spain",
    "Bangkok": "Bangkok, Thailand",
    "Rome": "Rome, Italy",
    "Hawaii": "Honolulu, Hawaii",
    "Greece": "Greece",
    "Egypt": "Egypt",
    "Mexico": "Cancun, Mexico",
    "Italy": "Rome, Italy",
    "Spain": "Barcelona, Spain",
    "Canada": "Vancouver, Canada",
    "Japan": "Tokyo, Japan",
    "Thailand": "Phuket, Thailand",
    "France": "Paris, France",
    "Australia": "Sydney, Australia",
    "Brazil": "Rio de Janeiro, Brazil",
    "South Korea": "Seoul, South Korea",
    "United Kingdom": "London, UK",
    "USA": "New York, USA",
    "Amsterdam": "Amsterdam, Netherlands"
}


In [60]:
data['Destination'] = data['Destination'].str.strip().str.title()

In [61]:
data['Destination'] = data['Destination'].replace(country_dict)

In [62]:
data['Transportation type'] = data['Transportation type'].replace({
    "Plane": "Flight",
    "Flight": "Flight",
    "Airplane": "Flight",
    "Subway": "Train",
    "Bus": "Bus",
    "Car rental": "Car",
    "Car Rental": "Car",
    "Ferry": "Ferry"
})

In [63]:
if data.iloc[128].isnull().all():  # Note: Indexing is zero-based, so row 128 is at index 127
    data.drop(index=128, inplace=True)

In [64]:
data.to_csv('cleaned_travel_data.csv', index=False)

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
df = pd.read_csv("Travel details dataset.csv")

In [67]:
df['Destination'] = df['Destination'].fillna('')
df['Transportation type'] = df['Transportation type'].fillna('')
df['Accommodation cost'] = df['Accommodation cost'].fillna('0')
df['Transportation cost'] = df['Transportation cost'].fillna('0')

In [68]:
def combine_features(row):
    return f"{row['Transportation type']} {row['Accommodation cost']} {row['Transportation cost']}"

df["combined_features"] = df.apply(combine_features, axis=1)

In [69]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])

In [70]:
cosine_sim = cosine_similarity(count_matrix)

In [71]:
def get_destination_from_index(index):
    return df[df.index==index]["Destination"].values[0]

def get_index_from_destination(destination):
    return df[df.Destination==destination].index.values[0]

In [96]:
# Example user preferences
user_transportation_type = "plane"
user_accommodation_cost = "2000"
user_transportation_cost = "500"

In [97]:
user_features = f"{user_transportation_type} {user_accommodation_cost} {user_transportation_cost}"

In [98]:
user_df = pd.DataFrame([user_features], columns=['combined_features'])
user_count_matrix = cv.transform(user_df['combined_features'])
user_cosine_sim = cosine_similarity(user_count_matrix, count_matrix)

In [99]:
similar_destinations = list(enumerate(user_cosine_sim[0]))
sorted_similar_destinations = sorted(similar_destinations, key=lambda x: x[1], reverse=True)

In [100]:
i = 0
for destination in sorted_similar_destinations:
    print(get_destination_from_index(destination[0]))
    i += 1
    if i >= 5:
        break

Cape Town
Bali
Bangkok
Japan
Mexico
